In [ ]:
!pip install alpha_vantage

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from alpha_vantage.timeseries import TimeSeries
import requests

# Function to retrieve stock data from Alpha Vantage API
def get_stock_data(symbol, api_key):
    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=symbol, outputsize='compact')
    return data

# Function to retrieve blockchain data from Blockchain.com API
def get_blockchain_data():
    url = "https://api.blockchain.com/v3/exchange/tickers/BTC-USD"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to retrieve blockchain data")
        return None

# Load your stock price data
alpha_vantage_api_key = '8C9OHXLIAQYBHC6G'
stock_data = get_stock_data('AAPL', alpha_vantage_api_key)

# Print stock data
print("Stock Data:")
print(stock_data)

# Load your blockchain data
blockchain_data = get_blockchain_data()

# Print blockchain data to understand its structure
print("\nBlockchain Data:")
print(blockchain_data)

# Assuming 'last_trade_price' is the relevant data point in blockchain_data
# Convert blockchain data to DataFrame
if blockchain_data is not None and 'last_trade_price' in blockchain_data:
    blockchain_df = pd.DataFrame(blockchain_data, index=[0])
    # Since 'timestamp' is not available, let's create a dummy timestamp
    blockchain_df['timestamp'] = pd.Timestamp.now().date()
    blockchain_df = blockchain_df.rename(columns={'timestamp': 'date'})
    print("\nBlockchain DataFrame:")
    print(blockchain_df)
else:
    print("No blockchain data available.")

# Merge stock data with blockchain data based on date
if 'date' in blockchain_df.columns:
    # Ensure that 'date' column in stock_data is of datetime64[ns] type
    stock_data['date'] = pd.to_datetime(stock_data.index)
    merged_data = pd.concat([stock_data.set_index('date'), blockchain_df.set_index('date')], axis=1, join='inner')

    if not merged_data.empty:
        # Preprocessing merged data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(merged_data.values)

        # Function to prepare data for algorithms
        def prepare_data(data, n_steps):
            X, y = [], []
            for i in range(len(data)):
                end_ix = i + n_steps
                if end_ix > len(data)-1:
                    break
                seq_x, seq_y = data[i:end_ix], data[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return np.array(X), np.array(y)

        # Define number of time steps for the algorithms
        n_steps = 10

        # Prepare data
        X, y = prepare_data(scaled_data, n_steps)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Initialize algorithms
        algorithms = {
            "Linear Regression": LinearRegression(),
            "Support Vector Regression": SVR(),
            "Random Forest Regression": RandomForestRegressor(),
            "Gradient Boosting Regression": GradientBoostingRegressor()
        }

        for name, model in algorithms.items():
            # Train model
            model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2])), y_train)

            # Make predictions
            predictions = model.predict(X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2])))

            # Inverse transform predictions and actual values
            predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
            actual_values = scaler.inverse_transform(y_test.reshape(-1, 1))

            # Calculate metrics
            rmse = np.sqrt(mean_squared_error(actual_values, predictions))
            mae = mean_absolute_error(actual_values, predictions)
            r2 = r2_score(actual_values, predictions)

            print(f"Algorithm: {name}")
            print(f"Root Mean Squared Error: {rmse}")
            print(f"Mean Absolute Error: {mae}")
            print(f"R-squared: {r2}")
            print()

            # Plot predictions vs actual values
            plt.figure(figsize=(10, 6))
            plt.plot(merged_data.index[-len(predictions):], predictions, label='Predictions')
            plt.plot(merged_data.index[-len(actual_values):], actual_values, label='Actual Values')
            plt.xlabel('Date')
            plt.ylabel('Price')
            plt.title(f'{name}: Predictions vs Actual Values')
            plt.legend()
            plt.show()

    else:
        print("Unable to merge data due to empty merged_data DataFrame.")
else:
    print("Unable to merge data due to missing date information.")


Stock Data:
            1. open  2. high    3. low  4. close   5. volume
date                                                        
2024-05-16  190.470  191.095  189.6601    189.84  52845230.0
2024-05-15  187.910  190.650  187.3700    189.72  70399988.0
2024-05-14  187.510  188.300  186.2900    187.43  52393619.0
2024-05-13  185.435  187.100  184.6200    186.28  72044809.0
2024-05-10  184.900  185.090  182.1300    183.05  50759496.0
...             ...      ...       ...       ...         ...
2023-12-29  193.900  194.400  191.7250    192.53  42672148.0
2023-12-28  194.140  194.660  193.1700    193.58  34049898.0
2023-12-27  192.490  193.500  191.0900    193.15  47899806.0
2023-12-26  193.610  193.890  192.8300    193.05  28919310.0
2023-12-22  195.180  195.410  192.9700    193.60  37149570.0

[100 rows x 5 columns]

Blockchain Data:
{'symbol': 'BTC-USD', 'price_24h': 66273.2, 'volume_24h': 0.91551158, 'last_trade_price': 65640.38}

Blockchain DataFrame:
    symbol  price_24h  volume_